In [ ]:
#from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import torch
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from langchain_text_splitters import RecursiveCharacterTextSplitter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train=pd.read_csv("../data/train.csv")

val=pd.read_csv("../data/val.csv")

test=pd.read_csv("../data/test.csv")

# Chunking steps
- We calculate number of tokens in document and query with BERT token estimator
- We update the max input length as max input length from the model - len of query - 1.
- We then use chunkipy TextChunker to chunk the document and store the chunks in a list.
- We add the query as prefix for each chunk.
- Do the process for all the queries and documents.

In [ ]:
from chunkipy import TextChunker, TokenEstimator
from transformers import AutoTokenizer

In [ ]:
train_s = train.sample(20, random_state = 42)
#train_s = train_s.loc[train_s['majority_label'] != 'Neutral']
eval_s = val.sample(5, random_state = 42)

label2id = {
    'Pro': 0,
    'Against': 1,
    'Neutral': 2,
    'Not-about': 3,
    'No majority': 4
}

# Create new column with label IDs
train_s['labels'] = train_s['majority_label'].map(label2id)
eval_s['labels'] = eval_s['majority_label'].map(label2id)

train['labels'] = train['majority_label'].map(label2id)
val['labels'] = val['majority_label'].map(label2id)

In [ ]:
class BertTokenEstimator(TokenEstimator):
    def __init__(self):
        self.bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

    def estimate_tokens(self, text):
        return len(self.bert_tokenizer.encode(text))


bert_token_estimator = BertTokenEstimator()

def estimate_tokens_wrapper(text):
    bert_token_estimator = BertTokenEstimator()
    return bert_token_estimator.estimate_tokens(text)


def get_chunks(text, input_length):
    text_chunker = TextChunker(input_length, tokens=True, token_estimator=BertTokenEstimator())
    return text_chunker.chunk(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train['Input2'] = (train['docTitle'] + " " + train['docCont']).astype(str)
test['Input2'] = (test['docTitle'] + " " + test['docCont']).astype(str)
val['Input2'] = (val['docTitle'] + " " + val['docCont']).astype(str)

In [ ]:
max_input_length = 512
train['chunks'] = ""
for index, row in train.iterrows():
    question = row['Query']
    text = row['Input2']
    q_len = estimate_tokens_wrapper(question)
    input_length = max_input_length - q_len - 1
    n_chunks = get_chunks(text, input_length)
    chunks = [question + " " + sent for sent in n_chunks]

    train.at[index, 'chunks'] = chunks


train = train.explode('chunks').reset_index(drop=True)
train['chunk_len'] = train['chunks'].apply(lambda x : len(x.split()))
train['chunk_tokens'] = train['chunks'].apply(estimate_tokens_wrapper)

train.to_csv('/content/drive/MyDrive/HLT/Data/chunked_train.csv')

Token indices sequence length is longer than the specified maximum sequence length for this model (3947 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (2103 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (7598 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1279 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1065 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

In [ ]:
max_input_length = 512
val['chunks'] = ""
for index, row in val.iterrows():
    question = row['Query']
    text = row['Input2']
    q_len = estimate_tokens_wrapper(question)
    input_length = max_input_length - q_len - 1
    n_chunks = get_chunks(text, input_length)
    chunks = [question + " " + sent for sent in n_chunks]

    val.at[index, 'chunks'] = chunks


val = val.explode('chunks').reset_index(drop=True)
val['chunk_len'] = val['chunks'].apply(lambda x : len(x.split()))
val['chunk_tokens'] = val['chunks'].apply(estimate_tokens_wrapper)

val.to_csv('/content/drive/MyDrive/HLT/Data/chunked_val.csv')

Token indices sequence length is longer than the specified maximum sequence length for this model (637 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (948 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (860 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3812 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (961 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

In [ ]:
max_input_length = 512
test['chunks'] = ""
for index, row in test.iterrows():
    question = row['Query']
    text = row['Input2']
    q_len = estimate_tokens_wrapper(question)
    input_length = max_input_length - q_len - 1
    n_chunks = get_chunks(text, input_length)
    chunks = [question + " " + sent for sent in n_chunks]

    test.at[index, 'chunks'] = chunks


test = test.explode('chunks').reset_index(drop=True)
test['chunk_len'] = test['chunks'].apply(lambda x : len(x.split()))
test['chunk_tokens'] = test['chunks'].apply(estimate_tokens_wrapper)

test.to_csv('/content/drive/MyDrive/HLT/Data/chunked_test.csv')

Token indices sequence length is longer than the specified maximum sequence length for this model (1161 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2605 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5624 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (553 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 